In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random

from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.utils import degree, scatter
from torch_geometric.nn.norm import GraphNorm
from torch_geometric.nn import MessagePassing

In [2]:
torch.__version__

'2.2.2'

In [2]:
# Check current default device
print(torch.get_default_device())

AttributeError: module 'torch' has no attribute 'get_default_device'

In [3]:
# torch.set_default_device('cuda')
# print(torch.get_default_device())

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
def preprocess_data(data):
    if not hasattr(data, 'pos') or data.pos is None:
        data.pos = torch.rand((data.num_nodes, 2)) * 100.0
    if not hasattr(data, 'atom_type') or data.atom_type is None:
        deg = degree(data.edge_index[0], num_nodes=data.num_nodes).long()
        data.atom_type = (deg % 200)
    if data.y is not None:
        data.y = data.y.float()
    else:
        data.y = torch.tensor([0.0])
    return data

In [5]:
class MLP(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout_rate=0.0, final_activation=None):
        super().__init__()
        layers = []
        prev = in_channels
        for hidden in hidden_channels:
            layers.append(nn.Linear(prev, hidden))
            layers.append(nn.ReLU())
            if dropout_rate > 0:
                layers.append(nn.Dropout(dropout_rate))
            prev = hidden
        layers.append(nn.Linear(prev, out_channels))
        if final_activation is not None:
            layers.append(final_activation)
        self.net = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.net(x)
    
    def reset_parameters(self):
        for m in self.net:
            if hasattr(m, 'reset_parameters'):
                m.reset_parameters()

In [6]:
class RGINConv(MessagePassing):
    def __init__(
        self,
        mlp_dims_node: list,
        mlp_dims_edge: list,
        aggr: str = 'sum',
        dropout_rate: float = 0.0,
    ):
        super().__init__(node_dim=0, aggr=aggr)
        self.node_dimses = mlp_dims_node
        self.edge_dimses = mlp_dims_edge
        self.aggr = aggr
        self.dropout_rate = dropout_rate
        
        self.node_mlp = nn.ModuleList([
            MLP(mlp_dims_node[0], mlp_dims_node[1:-1], mlp_dims_node[-1],
                final_activation=None, dropout_rate=dropout_rate)
            for _ in range(mlp_dims_edge[-1])
        ])
        self.edge_mlp = MLP(mlp_dims_edge[0], mlp_dims_edge[1:-1], mlp_dims_edge[-1],
                            dropout_rate=dropout_rate, final_activation=None)
        self.reset_parameters()
        
    def reset_parameters(self):
        for m in self.node_mlp:
            m.reset_parameters()
        self.edge_mlp.reset_parameters()
        
    @classmethod
    def from_config(cls, config):
        return cls(**config)
        
    def get_config(self):
        return {
            'mlp_dims_node': self.node_dimses,
            'mlp_dims_edge': self.edge_dimses,
            'aggr': self.aggr,
            'dropout_rate': self.dropout_rate,
        }
    
    def forward(self, x: torch.Tensor, edge_index, edge_attr: torch.Tensor, edge_weight=None):
        # Proses edge_attr melalui MLP dan softmax-kan di sepanjang dim terakhir
        edge_attr = self.edge_mlp(edge_attr).softmax(-1)
        if edge_weight is not None:
            edge_attr = edge_attr * edge_weight
        size = (x.size(0), x.size(0))
        out = torch.zeros(x.size(0), self.node_dimses[-1], device=x.device)
        
        for edge_idx in range(self.edge_dimses[-1]):
            h = self.propagate(edge_index, edge_attr=edge_attr, x=x, size=size, edge_idx=edge_idx)
            h2 = self.node_mlp[edge_idx](h)
            out = out + h2
        return out
    
    def message(self, x_j: torch.Tensor, edge_attr, edge_idx) -> torch.Tensor:
        return x_j * edge_attr[:, edge_idx].unsqueeze(-1)

In [7]:
def radius_interaction_graph(pos, batch, cutoff=10.0, max_neighbors=32):
    N, _ = pos.size()
    edge_index = [[], []]
    edge_weight = []
    for i in range(N):
        for j in range(N):
            if i == j:
                continue
            d = torch.norm(pos[i] - pos[j], p=2).item()
            if d <= cutoff:
                edge_index[0].append(i)
                edge_index[1].append(j)
                edge_weight.append(d)
    if len(edge_weight) == 0:
        edge_index = torch.tensor([[0], [0]], dtype=torch.long)
        edge_weight = torch.tensor([cutoff], dtype=torch.float)
    else:
        edge_index = torch.tensor(edge_index, dtype=torch.long)
        edge_weight = torch.tensor(edge_weight, dtype=torch.float)
    return edge_index, edge_weight

In [23]:
class DRGIN5(nn.Module):
    def __init__(self, 
                 node_dimses, 
                 edge_dimses, 
                 dropout_rate=0.0,
                 cutoff=10.0,
                 max_neighbors=32,
                 aggr: str = 'sum'):
        super().__init__()
        self.rgins = nn.ModuleList()
        self.norms = nn.ModuleList()
        self.atom_type_emb = nn.Embedding(200, node_dimses[0][0])
        self.cutoff = cutoff
        self.max_neighbors = max_neighbors
        self.dist_norm = GraphNorm(1)
        
        for mlp_dims_node, mlp_dims_edge in zip(node_dimses, edge_dimses):
            self.rgins.append(RGINConv(mlp_dims_node, mlp_dims_edge, dropout_rate=dropout_rate, aggr=aggr))
            self.norms.append(GraphNorm(mlp_dims_node[-1]))
            
        self.node_dimses = node_dimses 
        self.edge_dimses = edge_dimses 
        self.dropout_rate = dropout_rate
        self.aggr = aggr
        self.reset_parameters()
        
    def reset_parameters(self):
        for rgin in self.rgins:
            rgin.reset_parameters()
            
    @classmethod
    def from_config(cls, config):
        return cls(**config)
        
    def get_config(self):
        return {
            'node_dimses': self.node_dimses,
            'edge_dimses': self.edge_dimses,
            'dropout_rate': self.dropout_rate,
            'cutoff': self.cutoff,
            'max_neighbors': self.max_neighbors,
            'aggr': self.aggr,
        }
    
    def forward(self, data):
        edge_index, edge_weight = radius_interaction_graph(data.pos, data.batch,
                                                           cutoff=self.cutoff,
                                                           max_neighbors=self.max_neighbors)
        #edge_index=edge_index.to(device)
        #edge_weight=edge_weight.to(device)
        if data.batch is not None:
            print("edge_weight:",edge_weight)
            print("data.batch[edge_index[0]]:",data.batch[edge_index[0]])
            edge_attr = self.dist_norm(edge_weight.unsqueeze(-1), data.batch[edge_index[0]])
        else:
            edge_attr = self.dist_norm(edge_weight.unsqueeze(-1), torch.zeros_like(edge_index[0]).long())
        edge_weight = ((self.cutoff + 1) - edge_weight.unsqueeze(-1)) / (self.cutoff + 1)
        
        h = self.atom_type_emb(data.atom_type)
        
        for i, (rgin, norm) in enumerate(zip(self.rgins, self.norms)):
            h = rgin(h, edge_index, edge_attr, edge_weight=edge_weight)
            if i + 1 < len(self.rgins):
                h = norm(h, batch=data.batch)
                h = torch.tanh(h)
        if data.batch is not None:
            h = scatter(h, data.batch, dim=0, reduce='mean').mean(-1)
        else:
            h = h.mean((-2, -1))
        return h

In [ ]:
#1. download dat raw
2. process
3. taro di folder processed/

In [44]:
dataset = TUDataset(root='./data/TUDataset', name='BZR_MD')
short_dataset = dataset[:1]
print(f"Jumlah graph: {len(dataset)}")

Processing...


Jumlah graph: 306


Done!


In [39]:
TUDataset

torch_geometric.datasets.tu_dataset.TUDataset

In [31]:
#node feature -> atom type -> default value (0)
#edge feature -> interatomic distance -> default value (0)


In [42]:
import pandas as pd
df=pd.read_csv("./datasets/instagram/data.csv")

In [47]:
batch

DataBatch(edge_index=[2, 342], x=[19, 8], edge_attr=[342, 5], y=[1], batch=[19], ptr=[2])

In [48]:
batch.x

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0.]], device='cuda:0')

In [ ]:
batch

In [45]:
loader = DataLoader(short_dataset, batch_size=1, shuffle=True)

node_dimses = [[32, 64, 64]]
edge_dimses = [[1, 16, 3]]
model = DRGIN5(node_dimses, edge_dimses, dropout_rate=0.1, cutoff=10.0, max_neighbors=32, aggr='sum')

model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

model.train()
epochs = 1
for epoch in range(epochs):
    total_loss = 0
    for batch in loader:
        batch = batch.to(device)
        break
        optimizer.zero_grad()
        pred = model(batch)
        loss = criterion(pred, batch.y.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * batch.num_graphs
    avg_loss = total_loss / len(short_dataset)
    print(f"Epoch {epoch+1:02d} - Loss: {avg_loss:.4f}")

Epoch 01 - Loss: 0.0000


In [ ]:
0 - mean.index_select(0, batch) * self.mean_scale

In [ ]:
model.eval()
sample = next(iter(loader)).to(device)
with torch.no_grad():
    pred_sample = model(sample)
print("Prediksi nilai kontinu: ", pred_sample.cpu().numpy())

In [ ]:
def get_random_graph_loss(model, dataset, device, criterion):
    random_index = random.randint(0, len(dataset) - 1)
    data = dataset[random_index]
    
    data = data.to(device)
    
    model.eval()
    
    with torch.no_grad():
        output = model(data)
        
    loss = criterion(output, data.y.view(-1))
    
    print(f"Graph index: {random_index}")
    print(f"Output model untuk graph ini: {output.cpu().numpy()}")
    print(f"Loss untuk graph ini: {loss.item()}")